#### Sequential Learning App


In [31]:
#Imports

from resample.jackknife import resample as b_resample
from numpy import random
import ipywidgets as widgets
from ipywidgets import RadioButtons,FloatRangeSlider,Button,IntSlider
from IPython.display import display,Markdown
import pandas as pd
import sys
import statistics
from io import StringIO
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.spatial import distance_matrix

import time
import seaborn as sns

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn import preprocessing

#FUELS-imports
#from lolopy.learners import RandomForestRegressor
from operator import add
from sklearn.model_selection import train_test_split

from sklearn.manifold import TSNE
import seaborn as sns

from matplotlib.colors import Colormap

In [32]:
#Components

tab = widgets.Tab()    

out = widgets.Output(layout={'border': '1px solid black'})
out_plotting= widgets.Output(layout={'border': '1px solid black'})



out_algo=widgets.Output(layout={'border': '1px solid black'})
out_algo_alt=widgets.Output(layout={'border': '1px solid black'})


out_perform_experiment=widgets.Output(layout={'border': '1px solid black'})
out_input_space=widgets.Output(layout={'border': '1px solid black'})

out_res=widgets.Output(layout={'border': '1px solid black'})


up = widgets.FileUpload(accept="", multiple=False)

delim = widgets.RadioButtons(
    options=[';', ','],
    description='Separator: ',
    disabled=False)

delim_dec = widgets.RadioButtons(
    options=['.', ','],
    description='Decimal Delim: ',
    disabled=False)

eraser = widgets.SelectMultiple(
    options=['tab','"',"%"],
    value=['tab'],
    #rows=10,
    description='Eraser: ',
    disabled=False)

rows = widgets.IntSlider(
    value=0,
    step=1,
    description='# of lines:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d')

##ML-###pre processing

feature_selector=widgets.SelectMultiple(
    options=[],
    #tooltip("Select the Input Features")
    #rows=10,
    description='Features',
    disabled=False,
layout=widgets.Layout(width='50%'))
    
target_selection=widgets.SelectMultiple(
    options=[],
    #value='',
    placeholder='select the Target Variable',
    description='Target-Variable',
    disabled=False,
layout=widgets.Layout(width='50%'))

selector_plot_variable=widgets.SelectMultiple(
    options=[],
    #tooltip("Select the Input Features")
    #rows=10,
    description='Features',
    disabled=False,
layout=widgets.Layout(width='50%'))
    





graph_type = widgets.Dropdown(
    options=['Choose the Graph Type','Scatter', 'Scatter Matrix', 'Correlation Heatmap'],
    value='Choose the Graph Type',
    description='Chart Type:',
    disabled=False,
    layout=widgets.Layout(width='50%'))

x_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='X-Axis:',
    disabled=False)
y_axis = widgets.Dropdown(
    options=[''],
    value='',
    description='Y-Axis:',
    disabled=False)
color_picker = widgets.ColorPicker(
    concise=False,
    description='Color Picker: ',
    value='lightblue',
    disabled=False)
toggle = widgets.ToggleButtons(
    options=['Preview  ', 'Info  ', 'Stats  '],
    description='Options',
    disabled=False,
    button_style='warning',
    icons=['search', 'info', 'tachometer'])

select_x=widgets.Dropdown(
    options=[''],
    value='',
    placeholder='select X-Axis',
    description='X-Axis:',
    disabled=False,
layout=widgets.Layout(width='50%'))
select_y=widgets.Dropdown(
    options=[''],
    value='',
    placeholder='select Y-Axis',
    description='Y-Axis:',
    disabled=False,
layout=widgets.Layout(width='50%'))

select_hue=widgets.Dropdown(
    options=[''],
    value='',
    placeholder='select the hue',
    description='Hue:',
    disabled=False,
layout=widgets.Layout(width='50%'))

select_size=widgets.Dropdown(
    options=[''],
    value='',
    placeholder='select the Size',
    description='Size:',
    disabled=False,
layout=widgets.Layout(width='50%'))


select_strategy=widgets.Dropdown(
    options=['MEI','MU','MLI','MEID'],
    value='MEID',
    placeholder='select the strategy',
    description='Strategy:',
    disabled=False,
layout=widgets.Layout(width='50%'))

select_model=widgets.Dropdown(
    options=['Random Forrest (RF)','Decision Trees (DT)','Bagged Tree Ensembles (TE)','Gaussian Process Regression (GPR)'],
    value='Decision Trees (DT)',
    placeholder='select the Model',
    description='Model:',
    disabled=False,
layout=widgets.Layout(width='50%'))




button_confirm_strategy=widgets.Button(
    description='Confirm Strategy ',
    disabled=False,
    button_style='success',
    tooltip='Confirm selected Strategy',
    icon='check',
layout=widgets.Layout(width='50%',height ='inherit'))




#Options
box_features=widgets.VBox([])




#container_feature_options=widgets.VBox([widgets.HBox([box_features,box_features_slider]),box_confirm])
#feature_options= widgets.Accordion(children=[container_feature_options])

#feature_options.set_title(0,"Specify your Target")

In [33]:
#Buttons Anfang
button_upload = widgets.Button(
    description='Upload',
    disabled=False,
    button_style='success',
    tooltip='Click to Upload',
    icon='check',
layout=widgets.Layout(width='50%'))

button_preview = widgets.Button(
    description='Preview',
    disabled=False,
    button_style='info',
    tooltip='Click to Preview',
    icon='search',
layout=widgets.Layout(width='50%'))



#Plotting
button_plot = widgets.Button(
    description='Plot',
    disabled=False,
    button_style='warning',
    tooltip='Click to Plot',
    icon='pencil',
layout=widgets.Layout(width='100%'))




###Algos
button_confirm_feature=widgets.Button(
    description='Select Features ',
    disabled=False,
    button_style='success',
    tooltip='Confirm Feature Selection of Training Features',
    icon='check',
layout=widgets.Layout(width='50%',height ='inherit'))

button_confirm_target=widgets.Button(
    description='Select Target',
    disabled=False,
    button_style='success',
    tooltip='Confirm the selected Target Variable',
    icon='check',
layout=widgets.Layout(width='50%',height ='inherit'))

button_confirm_plot_var=widgets.Button(
    description='Confirm Target',
    disabled=False,
    button_style='success',
    tooltip='Confirm the selected Target Variable',
    icon='check',
    layout=widgets.Layout(width='50%',height ='inherit'))

button_plot_comparision=widgets.Button(
    description='Compare',
    disabled=False,
    button_style='warning',
    tooltip='Simplify the Columns',
    icon='fa-bar-chart',
    layout=widgets.Layout(width='50%')
)


button_confirm_options=Button(
    description='Confirm Options ',
    disabled=False,
    button_style='success',
    tooltip='Confirm options',
    icon='check',
    layout=widgets.Layout(width='50%',height ='inherit'))
                           
###Algos ende
##ML End

button_custom_strategey=widgets.Button(
    description='Customize',
    disabled=False,
    button_style='warning',
    tooltip='Simplify the Columns',
    icon='fa-bar-chart',
    layout=widgets.Layout(width='50%')
       
    
)
button_perform_experiment=widgets.Button(
    description='Run',
    disabled=False,
    button_style='warning',
    tooltip='Perform Experiment',
    icon='fa-calculator',
    layout=widgets.Layout(width='100%'))
    
#Buttons Ende

In [34]:
#FileUpload Tab
accordion = widgets.Accordion(children=[
    up, 
    widgets.HBox([delim, delim_dec, eraser]), 
    rows])

accordion.set_title(0, 'File Selection')
accordion.set_title(1, 'Delimiter')
accordion.set_title(2, 'Skip Rows')


accordion_box = widgets.VBox([
    accordion, 
    widgets.HBox([button_preview, button_upload ]),
    out
])

In [35]:
#Sequential Learning Tab 

quantile_tar_slider= widgets.FloatSlider(
    value=90,
    min=0,
    max=100,
    step=1,
    description='Quantile Target:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    
    layout=widgets.Layout(width='50%')
)


quantile_sample_slider= widgets.FloatSlider(
    value=50,
    min=0,
    max=100,
    step=5,
    description='Quantile Sample:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    
    layout=widgets.Layout(width='50%')
)
initial_sample_size_text=widgets.BoundedIntText(
    value=4,
    min=0,
    max=10,
    step=1,
    description='Initial Sample Size:',
    disabled=False,
    layout=widgets.Layout(width='50%')
)

#slider_quantile_container=widgets.VBox([widgets.HBox(children=[quantile_tar_slider,quantile_sample_slider,initial_sample_size_text])])

box_features_slider=widgets.VBox([])


DataPre_sl=widgets.VBox([
        widgets.HBox([feature_selector, button_confirm_feature ]),
        widgets.HBox([target_selection, button_confirm_target]),
#      slider_quantile_container,
#feature_options,
#button_split_data,
    box_features,
    quantile_tar_slider,
    quantile_sample_slider,
    initial_sample_size_text,
])

DataPre=widgets.VBox([
        widgets.HBox([feature_selector, button_confirm_feature ]),
        widgets.HBox([target_selection, button_confirm_target]),
       
    
])

iterations=widgets.IntSlider(
    value=30,
    min=0,
    max=100,
    step=5,
    description='Iterations:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    layout=widgets.Layout(width='50%')
)
show_results_button= widgets.Button(
    description='Show Results',
    disabled=False,
    button_style='warning',
    tooltip='Show Results',
    icon='fa-bar-chart',
    layout=widgets.Layout(width='100%')
)

custom_container=widgets.VBox([widgets.HBox([])])
results_container=widgets.VBox([widgets.HBox([])])

sl_settings= widgets.VBox([
    widgets.HBox([select_strategy, button_confirm_strategy ]),
    custom_container,
    widgets.HBox([select_model,iterations]),
    widgets.HBox([button_perform_experiment]),
    out_perform_experiment,
    out_input_space
   
    
    
])



sl_accordion=widgets.Accordion(children=[DataPre_sl,sl_settings])
sl_accordion.set_title(0,"Settings")
sl_accordion.set_title(1,"Sequential Learning Parameters")

In [36]:
#Plotting Tab



container_plot_options= widgets.VBox([])
button_container=widgets.HBox([button_plot])

plotting=widgets.VBox(children=[widgets.VBox( [
        widgets.HBox([graph_type]),
        container_plot_options,
        button_container,
        out_plotting
        
        
]


)])

children = [
    accordion_box, 
    widgets.VBox([toggle, out]),
    plotting,
    #plot_accordion,
    sl_accordion
    
   ]

In [37]:
#Tabs
tab.children = children
tab.set_title(0, "Upload")
tab.set_title(1, "Data Info")
tab.set_title(2, "Design Space Explorer")
tab.set_title(3, "Sequential Learning")

In [38]:
#EventHandling

def content_parser():
    if up.value == {}:
        """with out:
            out.clear_output
            display(Markdown('No CSV loaded'))
            #print('No CSV loaded')    """
    else:
        typ, content = "", ""
        up_value = up.value
        for i in up_value.keys():
            typ = up_value[i]["metadata"]["type"]
            if typ == "text/csv":
                content = up_value[i]["content"]
                content_str = str(content, 'utf-8')

                if eraser.value != {}: 
                    for val in eraser.value:
                        if val == "tab":
                            content_str = content_str.replace("\t","")
                        elif val =="%":
                            content_str = content_str.replace("\t","")
                        else:
                            content_str = content_str.replace(val,"")
                if content_str != "":
                    str_io = StringIO(content_str) 
                    return str_io
def df_converter():
    content = content_parser()
    if content is not None:
            df = pd.read_csv(content, sep=delim.value, index_col=False, skiprows=rows.value,decimal=delim_dec.value)
            df=df.apply(pd.to_numeric,errors="coerce")
            
            
            
                
            return df
    else:
        return None
def preview():
    
    df = df_converter()
    with out:
        out.clear_output()
        display(Markdown('Now this is how your DF looks like:'))
        
        if df is not None:
            print(df.head(10))
        else:
            display(Markdown('Configuration is wrong/missing...'))
            
def upload():
    
    df = df_converter()
    with out:
        out.clear_output()
        display(Markdown('Your uploaded DF looks like:'))
       
        if df is not None:
            print(df)
            x_axis.options = df.columns
            y_axis.options = df.columns
            feature_selector.options= df.columns
            select_x.options=df.columns
            select_y.options=df.columns
            select_size.options=df.columns
            select_hue.options=df.columns
            selector_plot_variable.options=df.columns
            
        else:
            display(Markdown('Configuration is wrong/missing...'))
            #print('Configuration is wrong/missing...')
def desc():
    info_level = toggle.value
    if info_level != {}:
        df = df_converter()
        with out:
            out.clear_output()
            print('\n ------Your {} looks like:------ \n'.format(
                info_level))
            if df is not None:
                if info_level == 'Info  ':
                    print(df.info(verbose=True))
                elif info_level == 'Stats  ':
                    print(df.describe())
                elif info_level == 'Preview  ':
                    print(df.head(5))
                else:
                    display(Markdown('Configuration is wrong/missing...'))
       
#plot methods   


def plot():
    graph = graph_type.value
    if graph=="Scatter":
        plot_scatter()
    elif graph=="Correlation Heatmap":
            plot_heat()
    elif graph=="Scatter Matrix":
            plot_pairwise()
          

        
def plot_pairwise():
    
    df =confirm_var()
    
    with out_plotting:
        out_plotting.clear_output()
        sns.pairplot(df)
        plt.show()

def plot_heat():
    
    df = confirm_var()
    
    with out_plotting:
        out_plotting.clear_output()
        corr = df.corr()

        plt.figure(figsize=(9,7))
        sns.heatmap(corr, annot=True, cmap='Blues')
        b, t = plt.ylim()
        plt.ylim(b+0.5, t-0.5)
        plt.title("Feature Correlation Heatmap")
        plt.show()
            
def plot_scatter():
    data=df_converter()
    with out_plotting:
        out_plotting.clear_output()
        fig, ax = plt.subplots(figsize=(10,7))
        #not generic
        sns.scatterplot(y=select_y.value, x=select_x.value, hue=select_hue.value, size=select_size.value, data=data, ax=ax, sizes=(50, 300))
        ax.set_title(select_y.value+ "vs"+ select_x.value)
        ax.legend(loc="upper left", bbox_to_anchor=(1,1))
        plt.show()
        plt.close(fig)      

In [39]:
#ml eventhandling
#pre pocessing

def confirm_options():
    items=box_features_slider.children
    df = df_converter()
    Y = df.loc[:,df.columns.isin(target_selection.value)]
    
    for slider in items:
            unt_grenz= slider.value[0]/100
            ob_grenz= slider.value[1]/100
            Y = Y[(Y >= Y.quantile(unt_grenz) ) & (Y <= Y.quantile(ob_grenz))]
            Y= Y.dropna()
    
    return Y   

#returns index of features
def confirm_features():
    df = df_converter()
    train = feature_selector.value
    
    target_selection.options=df.columns[~df.columns.isin(feature_selector.value)]
    
    train = df.columns[df.columns.isin(feature_selector.value)]
    
  
    return train
    
def confirm_var():
    df= df_converter()
    
    
    #df.columns[df.columns.isin(feature_selector.value)]
    #var = df.columns[df.columns.isin(selector_plot_variable.value)]
    
    selection = list(selector_plot_variable.value)
    
    var = df[selection]
 
    return var

    
#returns index of target    
def confirm_target():
    df = df_converter()
    #target_Var=target_selection.value
    #feature_selector.options= get_features()
    target = df.columns[df.columns.isin(target_selection.value)]
    
    
    if(len(box_features.children) == 0):
        create_dynamically_checkboxes()
   
    
    return target 
    
    

def create_dynamically_checkboxes():
    target= target_selection.value
    items = [RadioButtons(
    options=['maximize', 'minimize'],
    value='maximize', # Defaults to 'pineapple'
#    layout={'width': 'max-content'}, # If the items' names are long
    description=feature,
    disabled=False
) for feature in target]
    box_features.children=items

In [40]:
class sequential_learning:
    
    xlabel="Sequential Learning Iteration"
    dataframe = df_converter()
    init_sample_size=4
    number_of_executions=0
    features_df=df_converter()
    target_df=df_converter()
    targ_q=quantile_tar_slider.value
    samp_q=quantile_sample_slider.value
    min_distances_list=[]
    
    tries_list=[0]
    
    
    y_pred_dtr_mean=None
    y_pred_dtr_std=None
    y_pred_dtr=None
    SampIdx=None
    PredIdx=None
        
    def __init__(self,dataframe):  #constructor
        self.dataframe= dataframe
        
    
    def apply_feature_selection_to_df(self,dataframe):
        self.features_df = self.dataframe[confirm_features()]    
    
    def apply_target_selection_to_df(self,dataframe):
        self.target_df= self.dataframe[confirm_target()]     
        
    def normalize_data(self):
        
        
        dataframe_norm=(self.dataframe-self.dataframe.mean())/self.dataframe.std()
        target_df_norm=(self.target_df-self.target_df.mean())/self.target_df.std()
        features_df_norm=(self.features_df-self.features_df.mean())/self.features_df.std()
        
        
        self.features_df=features_df_norm
        self.target_df=target_df_norm
        self.dataframe=dataframe_norm
        return self.features_df, self.target_df, self.dataframe
        
    
    def set_init_sample_size(self):
        self.init_sample_size=initial_sample_size_text.value
    
        
    def set_number_of_ex(self):
        self.number_of_executions= iterations.value
        self.tries_list*self.number_of_executions
        
        
    def init_sampling(self):
        self.samp_q= quantile_sample_slider.value/100
        chosen_columns = confirm_target()
        
        
        #summ dataframe
        sum_ = self.dataframe[chosen_columns].sum(axis=1).to_frame()
        
        #quantile of DF 
        samp_q_t=sum_.quantile(self.samp_q)
        
        Index_label=np.where(sum_ < samp_q_t )
        Index_label=Index_label[0]
        
        init_sample_set = np.ones((0,self.init_sample_size))
        
        
        for i in range(self.number_of_executions):
                    
                    init_sample_set=np.vstack([init_sample_set, random.choice(Index_label,self.init_sample_size)])
       
        return init_sample_set
    
    def decide_max_or_min(self):
        
        max_or_min_optimizations_list=[]
        
        for decide in range(len(self.target_df.columns)):
            
                            
            max_or_min_optimizations_list.append(box_features.children[decide].value)
        
        return max_or_min_optimizations_list
            
    def start_sequential_learning(self):
        
        distances=[]
        current_distances_list=[0]
        with out_perform_experiment:
                display(Markdown('Sequential Learning is running...'))
       
        
            
        init_sample_set=self.init_sampling()
       
        
        for i in range(self.number_of_executions):
            
            current_distances_list=[0]
            
            self.SampIdx=init_sample_set[i].astype(int)
            self.PredIdx=self.dataframe
            self.PredIdx = self.PredIdx.drop(self.PredIdx.index[self.SampIdx]).index
            
            
            
            
            if select_model.value == 'Random Forrest (RF)':
                   self.y_pred_dtr,self.y_pred_dtr_mean, self.y_pred_dtr_std =  self.fit_RF_wJK()
            elif select_model.value == 'Decision Trees (DT)':
                    self.y_pred_dtr,self.y_pred_dtr_mean, self.y_pred_dtr_std =  self.fit_DT_wJK()
            elif select_model.value == 'Bagged Tree Ensembles (TE)':
                    self.y_pred_dtr,self.y_pred_dtr_mean, self.y_pred_dtr_std =  self.fit_TE_wJK()
            elif select_model.value == 'Gaussian Process Regression (GPR)':
                    print("GPR")
                
            self.tries_list.append(self.init_sample_size)
            decide_optimization=self.decide_max_or_min()
           
            for k in decide_optimization:
                index_of_target = decide_optimization.index(k)
                
                if (k == "minimize"): 
                    self.target_df.iloc[:,index_of_target]*(-1)
                    
                    
            #set quantile
            self.targ_q = quantile_tar_slider.value/100
            
            
            #targ_q_t= self.target_df.quantile(self.targ_q)
            sum_ = self.dataframe[confirm_target()].sum(axis=1).to_frame()
            targ_q_t= sum_.quantile(self.targ_q)
            while sum_.loc[self.SampIdx].max().values <= targ_q_t.values:
                            #Update_Idx_with_Strategy 

                            if confirm_strategy()=='MEI':
                                      self.updateIndexMEI()

                            elif confirm_strategy()=='MU':
                                    self.updateIndexMU()

                            elif confirm_strategy()=='MLI':
                                   self.updateIndexMLI()
                            elif confirm_strategy()=='MEID':
                                   self.updateIndexMEID() 

                            ##Train Model

                            if select_model.value == 'Random Forrest (RF)':
                                self.y_pred_dtr,self.y_pred_dtr_mean, self.y_pred_dtr_std =  self.fit_RF_wJK()
                            elif select_model.value == 'Decision Trees (DT)':
                                self.y_pred_dtr,self.y_pred_dtr_mean, self.y_pred_dtr_std =  self.fit_DT_wJK()
    
                            elif select_model.value == 'Bagged Tree Ensembles (TE)':
                                self.y_pred_dtr,self.y_pred_dtr_mean, self.y_pred_dtr_std =  self.fit_TE_wJK()

                            elif select_model.value == 'Gaussian Process Regression (GPR)':
                                print("GPR")

                            schwellwert=sum_.quantile(self.targ_q)
                            Index_=np.where(sum_ >= schwellwert )
                            Index_=Index_[0]

                            
                            distance= distance_matrix(self.dataframe.loc[self.SampIdx],self.dataframe.iloc[Index_])
                            #distance= distance_matrix(self.dataframe.loc[self.SampIdx],self.dataframe.iloc[self.PredIdx])
                            
                            #print("Distance Matrix: ",distance)
                           
                            distance=distance.min()
                            current_distances_list.append(distance)

                            
                            self.tries_list[i]=self.tries_list[i]+1
                            
                            
            
            
                        
                        
            distances.append(current_distances_list)
        
                
        
        ##Distance Plot
        
            with out_perform_experiment:
                fig1 = plt.figure(figsize=(10, 6))

                ax3 = plt.axes()
                ax3.set_title('Distance to target for each SL run')
                plt.xlabel('Number of SL iterations')
                plt.ylabel("Distance to target")

                for runs in range(len(distances)):
                    ax3.plot(distances[runs], color='k')
                
                #plt.close(fig1)
 

            display(Markdown('Sequential Learning performed ✅'))
            
            with out_perform_experiment:
                out_perform_experiment.clear_output(wait=True)
                time.sleep(1.0)
                fig2=plt.figure(figsize=(10, 6))
                plt.xlabel('Number of required Experiments')
                plt.ylabel("Frequency")
                plt.title("Performance histogram for %s with strategy %s -  required experiments"%(select_model.value,select_strategy.value))

                plt.hist(self.tries_list,range(self.number_of_executions))
                plt.show()
                #plt.close(fig2)
                        

           
        
        self.plot_TSNE_input_space()
    
        
            

    
    def updateIndexMEI(self):
            
            index_max = np.argmax(self.y_pred_dtr_mean)
            new_SampIdx=np.append(self.SampIdx,self.PredIdx[index_max])
            self.SampIdx=new_SampIdx
            new_PredIdx = np.delete(self.PredIdx, index_max)
            self.PredIdx=new_PredIdx
            
            
    def updateIndexMEID(self):
            schwellwert=np.quantile(self.y_pred_dtr_mean,0.95)
            Index_=np.where(self.y_pred_dtr_mean>=schwellwert )
            Index_=Index_[0]
            
            
            
            distance= distance_matrix(self.dataframe.loc[self.SampIdx],self.dataframe.iloc[Index_])
            min_distances=distance.min(1)
            result = np.where(distance == min_distances.max())
            print('Tuple of arrays returned : ', result)
            print('List of coordinates of minimum value in Numpy array : ')
            # zip the 2 arrays to get the exact coordinates
            listOfCordinates = list(zip(result[0], result[1]))
            print("Index_",Index_)
            print("Index_[result[1]]",Index_[result[1]])
            
            
            
            index_max=Index_[result[1]]
            
            
            
            new_SampIdx=np.append(self.SampIdx,self.PredIdx[index_max])
            self.SampIdx=new_SampIdx
            new_PredIdx = np.delete(self.PredIdx, index_max)
            self.PredIdx=new_PredIdx
            
            
            
    def updateIndexMU(self):
            index_max = np.argmax(self.y_pred_dtr_std)
            new_SampIdx=np.append(self.SampIdx,self.PredIdx[index_max])
            self.SampIdx=new_SampIdx
            new_PredIdx = np.delete(self.PredIdx, index_max)
            self.PredIdx=new_PredIdx
            
            
    def updateIndexMLI(self):
            
            index_max = np.argmax(self.y_pred_dtr_mean+self.y_pred_dtr_std)
            new_SampIdx=np.append(self.SampIdx,self.PredIdx[index_max])
            self.SampIdx=new_SampIdx
            new_PredIdx = np.delete(self.PredIdx, index_max)
            self.PredIdx=new_PredIdx
                          
    def plot_TSNE_input_space(self):
        
        
            X=self.features_df
            y=self.target_df

            tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
            tsne_results = tsne.fit_transform(X)


            with out_input_space:
                        # Plot Results in reduced FS
                        out_input_space.clear_output(wait=True)
                        fig3= plt.figure(figsize=(10, 6))
                        
                        color = [y.sum(axis=1)] # y sind hier die labels
                        sc=plt.scatter(x=tsne_results[:,0], y=tsne_results[:,1], s=100, c=color)
                        palette=sns.color_palette("hls")
                        plt.colorbar(sc)
                        plt.title("Design space in TSNE-coordinates")
                        plt.show()
                        
                        plt.close(fig3)
                        
                        

                
    def fit_DT_wJK(self):        
       
        td= []
        for x in b_resample(self.features_df.iloc[self.SampIdx]):
            td.append(x)
        tl=[]
        for x in b_resample(self.target_df.iloc[self.SampIdx]): 
            tl.append(x)
            
        
        td=np.array(td)
        tl=np.array(tl)
        self.y_pred_dtr=[]
        for i in range(len(td)):
            dtr = DecisionTreeRegressor()
            dtr.fit(td[i], tl[i])
            
            self.y_pred_dtr.append(dtr.predict(self.features_df.iloc[self.PredIdx]))
        
        self.y_pred_dtr=np.array(self.y_pred_dtr)
        #quick debug
        if(confirm_strategy()=="MEID"):
            row_means = self.y_pred_dtr.mean(axis=0)
            row_std = self.y_pred_dtr.std(axis=0)
        else:
            self.y_pred_dtr=self.y_pred_dtr.T
            row_means = self.y_pred_dtr.mean(axis=1)
            row_std = self.y_pred_dtr.std(axis=1)
        
        return self.y_pred_dtr,row_means, row_std
        
    def fit_TE_wJK(self):
        
        td= []
        for x in b_resample(self.features_df.iloc[self.SampIdx]):
            td.append(x)
        tl=[]
        for x in b_resample(self.target_df.iloc[self.SampIdx]): 
            tl.append(x)
            
        
        td=np.array(td)
        tl=np.array(tl)
        self.y_pred_dtr=[]
        for i in range(len(td)):
            dtr = BaggingRegressor()
            dtr.fit(td[i], tl[i])
            
            self.y_pred_dtr.append(dtr.predict(self.features_df.iloc[self.PredIdx]))
        
        self.y_pred_dtr=np.array(self.y_pred_dtr)
        self.y_pred_dtr=self.y_pred_dtr.T
        row_means = self.y_pred_dtr.mean(axis=1)
        row_std = self.y_pred_dtr.std(axis=1)
        
        return self.y_pred_dtr,row_means, row_std
        
        
    def fit_RF_wJK(self):
        
        td= []
        for x in b_resample(self.features_df.iloc[self.SampIdx]):
            td.append(x)
        tl=[]
        for x in b_resample(self.target_df.iloc[self.SampIdx]): 
            tl.append(x)
            
        
        td=np.array(td)
        tl=np.array(tl)
        self.y_pred_dtr=[]
        for i in range(len(td)):
            dtr = RandomForestRegressor()
            dtr.fit(td[i], tl[i])
            
            self.y_pred_dtr.append(dtr.predict(self.features_df.iloc[self.PredIdx]))
        
        self.y_pred_dtr=np.array(self.y_pred_dtr)
        self.y_pred_dtr=self.y_pred_dtr.T
        row_means = self.y_pred_dtr.mean(axis=1)
        row_std = self.y_pred_dtr.std(axis=1)
        
        return self.y_pred_dtr,row_means, row_std

        
        
    def main(self):
        self.apply_feature_selection_to_df(self.dataframe)
        self.apply_target_selection_to_df(self.dataframe)
        self.normalize_data()
        self.set_init_sample_size()
        self.set_number_of_ex()
        init_sample_set=self.init_sampling()
        
        self.start_sequential_learning()
        
        try:
            self.start_sequential_learning()
        except IndexError as e:
            with out_perform_experiment:
                display(Markdown("Sth went wrong ❗️Try to restart the App 💡"))
        
    
        
                
        
        


In [41]:
text = widgets.Text(value='Hello World',
            placeholder='',
            description='Customize Strategy:',
            disabled=False)

customize = Button(description='Customize',
                    disabled=False,
                    button_style='warning',
                    tooltip='Click to Confirm Customization',
                    icon='fa-wrench ',
                    layout=widgets.Layout(width='50%'))

def confirm_strategy():
    strategy= select_strategy.value
    if strategy != {}:
        if strategy =='Customize Strategy':
            custom_container.children=[text,customize]
        else:
            custom_container.children=[]
            return select_strategy.value
                
                
                
def add_custom_to_strategy():
    select_strategy.options.append(text.value)

            
        
    

In [42]:
df=df_converter()
s= sequential_learning(df)


In [43]:
######EventListener######### 

###Uploads Anfang####


def preview_clicked(b):
    try:
        preview()
    except pd.errors.ParserError:
        with out:
            out.clear_output()
            display(Markdown("Sth. wennt wrong! Please check your csv and the upload settings"))
    
def upload_clicked(b):
    try:
        upload()
    except pd.errors.ParserError:
        with out:
            out.clear_output()
            display(Markdown("Sth. wennt wrong! Please check your csv and the upload settings"))
def desc_clicked(b):
    desc()

    
###Uploads Ende####


###Plot Anfang####
def plotter_clicked(b):
    plot()

def pairwise_clicked(b):
    plot_pairwise()
def heat_clicked(b):
    plot_heat()
def scatter_clicked(b):
    plot_scatter()
    
###Plot Ende####

    
###Selection SL Anfang###

def confirm_features_clicked(b):
    confirm_features()
    
def confirm_target_clicked(b):
    confirm_target()
    
def on_graph_type_change(change):
        
    if graph_type.value =="Scatter":
        container_plot_options.children= [widgets.HBox([select_x,select_y]),
        widgets.HBox([select_hue,select_size])]
    elif graph_type.value =="Scatter Matrix":
        
        container_plot_options.children= [widgets.HBox([selector_plot_variable,button_confirm_plot_var])]
                                          
    elif graph_type.value =='Correlation Heatmap':
        container_plot_options.children= [widgets.HBox([selector_plot_variable,button_confirm_plot_var])]
        
    else: container_plot_options.children=[]


def confirm_var_clicked(b):
    confirm_var()

def confirm_options_clicked(b):
    confirm_options()
    
def customize_clicked(b):
    add_custom_to_strategy()
def confirm_strategy_clicked(b):
    confirm_strategy()
    
def perform_experiment_clicked(b):
    s= sequential_learning(df_converter())
    s.main()
    
    
def on_value_change(change):
    quantile_sample_slider.max=quantile_tar_slider.value



In [44]:
#Eventss
#plot

graph_type.observe(on_graph_type_change,names='value')


button_preview.on_click(preview_clicked)


button_confirm_plot_var.on_click(confirm_var_clicked)


quantile_tar_slider.observe(on_value_change,names='value')


#show_results_button.on_click(show_results_button_clicked)

button_confirm_feature.on_click(confirm_features_clicked)
button_confirm_target.on_click(confirm_target_clicked)

#sl


button_confirm_options.on_click(confirm_options_clicked)

button_confirm_strategy.on_click(confirm_strategy_clicked)

customize.on_click(customize_clicked)


button_perform_experiment.on_click(perform_experiment_clicked)



#upload etc

button_upload.on_click(upload_clicked)
toggle.observe(desc_clicked, 'value')

button_plot.on_click(plotter_clicked)

In [45]:
tab

Tuple of arrays returned :  (array([0]), array([10]))
List of coordinates of minimum value in Numpy array : 
Index_ [ 13  35  38  48  49  71  74 103 104 105 129 130 131 132 139 140 141 169
 170 171]
Index_[result[1]] [129]
Tuple of arrays returned :  (array([0]), array([15]))
List of coordinates of minimum value in Numpy array : 
Index_ [ 13  34  35  38  48  49  70  71  74 100 101 103 104 105 106 129 130 131
 138 139 168 169]
Index_[result[1]] [129]
Tuple of arrays returned :  (array([2]), array([21]))
List of coordinates of minimum value in Numpy array : 
Index_ [ 35  36  37  38  39  40  41  71  72  73  74  75  76  77 104 105 129 130
 131 132 133 157 158 159 160 161 162 163 187 188 189 190 191 192 193]
Index_[result[1]] [157]
Tuple of arrays returned :  (array([1]), array([4]))
List of coordinates of minimum value in Numpy array : 
Index_ [ 34  35  38  42  49  70  71  74  78 100 101 102 103 104 105 106 107 108
 129 130 131 132 157 166 167]
Index_[result[1]] [49]
Tuple of arrays return

Sequential Learning performed ✅

Tuple of arrays returned :  (array([2]), array([5]))
List of coordinates of minimum value in Numpy array : 
Index_ [ 35  36  37  38  43  69  70  71  72  77 101 102 103 104 105 106 108 109
 129 130 131 132 133 134 135 159 160 161 162 189]
Index_[result[1]] [69]


Sequential Learning performed ✅

Tuple of arrays returned :  (array([2]), array([12]))
List of coordinates of minimum value in Numpy array : 
Index_ [ 14  15  16  17  23  34  49  50  51  52  58  70  88  93  94  95 103 104
 105 117 118 120 129 130 146 147 176 177]
Index_[result[1]] [88]
Tuple of arrays returned :  (array([3]), array([8]))
List of coordinates of minimum value in Numpy array : 
Index_ [ 18  19  20  21  53  54  55  56  88 113 114 115 117 118 142 143 144 146
 147 172 173 174 176 177]
Index_[result[1]] [88]
Tuple of arrays returned :  (array([5]), array([2]))
List of coordinates of minimum value in Numpy array : 
Index_ [ 18  19  20  21  29  30  31  32  41  42  43  44  53  54  55  56  65  66
  67  68  77  78  79  80  97 107 116 117 125 126 135 136 145 146 155 156
 165 166 175 176 185 186 195 196]
Index_[result[1]] [20]
Tuple of arrays returned :  (array([6]), array([17]))
List of coordinates of minimum value in Numpy array : 
Index_ [ 15  49  96 106 114 115 116 123 124 125 133 134 135 137 138 143 144 145
 1

...

Sequential Learning performed ✅

Sequential Learning performed ✅

Sequential Learning performed ✅

Sequential Learning performed ✅

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 203 samples in 0.000s...
[t-SNE] Computed neighbors for 203 samples in 0.002s...
[t-SNE] Computed conditional probabilities for sample 203 / 203
[t-SNE] Mean sigma: 1.797381
[t-SNE] KL divergence after 250 iterations with early exaggeration: 56.482826
[t-SNE] KL divergence after 300 iterations: 0.335303
